3. Exploratory Data Analysis  
3.1 Imports & loading Data  
3.2 Explore the data  
   * a. Features scanning & evaluation  
   * b. Visualization  
   * c. PCA tranformation if needed
* 3.3 Conclusion & summary

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
GunVio20132018 = pd.read_csv('../data/Clean_data/GunViolence2013_2018.csv',index_col='date')

In [4]:
GunVio20132018_final = pd.read_csv('../data/Clean_data/GunViolence2013_2018_final.csv',index_col='date')

C:\Users\Binhkn\anaconda3\envs\DS\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (4,6,8,14,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
GunVio20132018.info()

<class 'pandas.core.frame.DataFrame'>
Index: 239677 entries, 2013-01-01 to 2018-03-31
Data columns (total 37 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   incident_id                  239677 non-null  int64  
 1   state                        239677 non-null  object 
 2   city_or_county               239677 non-null  object 
 3   address                      223180 non-null  object 
 4   n_killed                     239677 non-null  int64  
 5   n_injured                    239677 non-null  int64  
 6   incident_url                 239677 non-null  object 
 7   source_url                   239209 non-null  object 
 8   incident_url_fields_missing  239677 non-null  bool   
 9   congressional_district       227733 non-null  float64
 10  gun_stolen                   140179 non-null  object 
 11  gun_type                     140226 non-null  object 
 12  incident_characteristics     239351 non-null  obje

In [5]:
GunVio20132018_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 366148 entries, 2013-01-19 to 2018-02-14
Data columns (total 64 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Title                        366148 non-null  object 
 1   Location                     366148 non-null  object 
 2   Area                         366148 non-null  object 
 3   Incident Area                366148 non-null  object 
 4   Target                       366148 non-null  object 
 5   Cause                        366148 non-null  object 
 6   Summary                      366148 non-null  object 
 7   Shooter status               366148 non-null  object 
 8   No. of shooter/suspect       366148 non-null  object 
 9   Fatalities_x                 366148 non-null  int64  
 10  Injured                      366148 non-null  int64  
 11  Total victims                366148 non-null  int64  
 12  Policeman Killed             366148 non-null  int6

In [8]:
a = GunVio20132018.columns
b = GunVio20132018_final.columns

In [9]:
print(a, b)

Index(['incident_id', 'state', 'city_or_county', 'address', 'n_killed',
       'n_injured', 'incident_url', 'source_url',
       'incident_url_fields_missing', 'congressional_district', 'gun_stolen',
       'gun_type', 'incident_characteristics', 'latitude',
       'location_description', 'longitude', 'n_guns_involved', 'notes',
       'participant_age', 'participant_age_group', 'participant_gender',
       'participant_name', 'participant_relationship', 'participant_status',
       'participant_type', 'sources', 'state_house_district',
       'state_senate_district', 'date.1', 'year', 'month', 'monthday',
       'weekday', 'total loss', 'participant_type_map', 'participant_age_map',
       'participant_gender_map'],
      dtype='object') Index(['Title', 'Location', 'Area', 'Incident Area', 'Target', 'Cause',
       'Summary', 'Shooter status', 'No. of shooter/suspect', 'Fatalities_x',
       'Injured', 'Total victims', 'Policeman Killed', 'Age',
       'Employeed (Y/N)', 'Employed at'